In [1]:
import os
import warnings
import logging
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

#sklearn - basic ML tools
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
from sklearn import metrics

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Conv3D, MaxPool3D, BatchNormalization, Flatten, LSTM, Dense, TimeDistributed, Input, Dropout, GlobalAveragePooling1D, GlobalAveragePooling3D, MaxPooling3D
from tensorflow.keras import Sequential, optimizers, Model
from tensorflow.keras.callbacks import CSVLogger
from keras import utils
from keras.utils import to_categorical, plot_model

#nilearn - neuroimaging tailored library
import nilearn
from nilearn import datasets
from nilearn.maskers import NiftiMasker
from nilearn import plotting
import nibabel as nib

from scipy import ndimage
from scipy.ndimage import zoom

warnings.filterwarnings("ignore")

2023-10-01 09:47:02.917816: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-01 09:47:02.917899: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-01 09:47:02.919578: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-01 09:47:04.434746: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
gpu = tf.config.experimental.list_physical_devices('GPU')
print("GPU Available: ", gpu)

GPU Available:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2023-10-01 09:47:20.349514: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-01 09:47:20.425536: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-01 09:47:20.428445: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [3]:
# Import preprocessed, ready-to-go, datasets
adhd_data=datasets.fetch_adhd(n_subjects=40, resume=False)
adhd_data.keys()

dict_keys(['func', 'confounds', 'phenotypic', 'description'])

### EDA

In [43]:
nilearn.datasets.get_data_dirs(data_dir=None)

['/home/jupyter/nilearn_data']

In [ ]:
pd.set_option('display.max_columns', None)
pd.read_csv('/home/jupyter/nilearn_data/adhd/ADHD200_40subs_motion_parameters_and_phenotypics.csv', sep=',')

In [44]:
pd.read_csv('/home/jupyter/nilearn_data/adhd/ADHD200_40subs_slice_timing_parameters.csv', sep=',')

,Site,TR (seconds),Reference,Acquisition
0,Brown,2.00,17,alt+z
1,KKI,2.50,24,seq+z
2,NeuroImage,1.96,19,alt+z
3,NYU,2.00,17,alt+z
4,OHSU,2.50,18,alt+z2
5,Peking_1,2.00,17,alt+z
6,Peking_2,2.00,17,alt+z
7,Peking_3,2.00,15,alt+z2
8,Pittsburgh,1.50,15,alt+z
9,WashU,2.50,16,alt+z2


In [ ]:
ls /home/jupyter/nilearn_data/adhd 

In [ ]:
with open("/home/jupyter/nilearn_data/adhd/ADHD200_40subs_ID.txt") as f: 
     print (f.read())

In [ ]:
ls /home/jupyter/nilearn_data/adhd/data/7774305

In [ ]:
adhd_data.phenotypic[0]['adhd']

In [ ]:
adhd_data.confounds[0]

In [45]:
pd.read_csv(adhd_data.confounds[1], sep='\t')

,csf,constant,linearTrend,wm,global,motion-pitch,motion-roll,motion-yaw,motion-x,motion-y,motion-z,gm,compcor1,compcor2,compcor3,compcor4,compcor5
0,13343.032102,1.0,0.0,9486.199546,9921.655259,0.2691,0.0374,-0.0041,-0.0290,-0.0638,-0.0429,9540.217163,-0.029119,0.027238,-0.068965,0.053021,-0.079315
1,13329.224068,1.0,1.0,9497.003325,9925.703297,0.2492,0.0374,-0.0049,-0.0363,-0.0493,-0.0556,9547.195835,0.011274,-0.034316,-0.171048,0.069540,-0.072535
2,13291.755627,1.0,2.0,9484.012965,9895.596376,0.2371,0.0424,-0.0155,-0.0248,-0.0532,-0.0308,9524.844100,0.018361,0.013027,-0.032447,0.132613,-0.116010
3,13275.211268,1.0,3.0,9478.347686,9885.200123,0.2436,0.0353,-0.0031,-0.0263,-0.0566,-0.0348,9508.965221,-0.013877,-0.094640,0.018219,0.117555,-0.123433
4,13279.199963,1.0,4.0,9485.204640,9880.712964,0.2409,0.0362,-0.0185,-0.0281,-0.0588,-0.0323,9514.826901,0.024742,0.010994,-0.010686,0.068357,-0.059298
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,13404.753750,1.0,171.0,9637.143270,10087.887458,-0.2936,-0.0454,-0.0468,0.0067,0.1180,0.0204,9723.878198,-0.018102,0.144588,-0.022891,0.044318,0.043382
172,13443.144519,1.0,172.0,9639.132354,10096.289703,-0.2743,-0.0480,-0.0390,0.0208,0.1098,0.0158,9725.204707,-0.059910,0.020106,-0.043166,0.080873,-0.024700
173,13452.822209,1.0,173.0,9645.685284,10088.448834,-0.2953,-0.0399,-0.0320,0.0127,0.1055,0.0340,9714.818066,0.063437,-0.041775,-0.111803,0.114312,0.108542
174,13434.011000,1.0,174.0,9633.262418,10079.278830,-0.2804,-0.0392,-0.0428,0.0332,0.1166,0.0324,9710.930758,0.024752,0.030090,-0.046833,0.076331,0.042856


In [ ]:
func_filenames = adhd_data.func
confounds = adhd_data.confounds
phenotypic = adhd_data.phenotypic

In [46]:
nilearn.datasets.get_data_dirs(data_dir=phenotypic)

['(\'"18"\', 9750701, \'"rest_1"\', 0.0654, 0, 0.2557, 0.1026, 0., 2.1544, 1.3514, \'"NYU"\', \'NA\', \'"data_set"\', 10.74, \'"M"\', \'"0.91"\', \'NA\', \'113\', \'103\', \'121\', \'NA\', 1, 0, \'0\', \'0\', \'0\', \'""\', \'""\', \'"pass"\', \'39\', \'40\', \'43\', \'45\', \'44\', \'45\', \'48\', \'41\', \'41\', \'41\', \'41\', \'40\', \'43\', \'40\', \'NA\', \'"pass"\', \'"open"\', \'""\', \'""\', \'""\', \'""\', \'NA\', \'NA\', \'NA\', \'NA\', \'NA\', \'NA\', \'"pass"\', \'NA\', \'NA\', \'NA\', \'NA\', \'NA\', \'NA\', \'"yes"\', \'""\')']

In [ ]:
# %load (func_filenames[0])
/home/jupyter/nilearn_data/adhd/data/0010042/0010042_rest_tshift_RPI_voreg_mni.nii.gz

### data consumption

In [ ]:
ls /home/jupyter/nilearn_data/adhd/data/

In [47]:
a = nib.load('/home/jupyter/nilearn_data/adhd/data/0027037/0027037_rest_tshift_RPI_voreg_mni.nii.gz')

In [48]:
a.get_fdata().shape

(61, 73, 61, 261)

In [ ]:
phenotypic['adhd']

In [ ]:
glob('/home/jupyter/nilearn_data/adhd/data/*/*.nii.gz')

In [ ]:
ls 

In [ ]:
from glob import glob

IMAGE_PATH_LIST = glob('/home/jupyter/nilearn_data/adhd/data/*/*.nii.gz')
#DATA = tf.data.Dataset.list_files(IMAGE_PATH_LIST)

#PHENO_PATH_LIST = glob('/home/jupyter/nilearn_data/adhd/data/*/*regressors.csv')
#PHENO = tf.data.Dataset.list_files(PHENO_PATH_LIST)

In [40]:
from os import getcwd, mkdir, path

write_dir = path.join(getcwd(), 'results')
if not path.exists(write_dir):
    mkdir(write_dir)

In [41]:
write_dir

'/home/jupyter/results'

In [42]:
from nilearn.image import mean_img

mean_img_ = mean_img(concat_img)

In [4]:
labels = []  # 1 if ADHD, 0 if control
img_file = []
#mask_files = []
img_label = []

for func_file, confound_file, phenotypic in zip(
        adhd_data.func, adhd_data.confounds, adhd_data.phenotypic):
    
    #mask_files.append(mask)
    img_file.append(func_file)
    labels.append(phenotypic['adhd'])
    img_label.append((func_file, phenotypic['adhd']))
    fmri_img = image.concat_imgs([fmri_filenames,fmri_filenames])

In [66]:
list_concats[0]

In [83]:
sera = nilearn.image.iter_img(list_concats)

In [70]:
single_mni_image = index_img(list_concats[0], 1)
print(single_mni_image.shape)

(61, 73, 61)


In [73]:
concat_img.shape

(61, 73, 61, 176)

In [74]:

first_three_images = index_img(concat_img,slice(0, 3))
print(first_three_images.shape)

(61, 73, 61, 3)


In [84]:
sera

<generator object _iter_check_niimg at 0x7ff33e78c120>

In [91]:
from nilearn.image import iter_img

for i, cur_img in enumerate(iter_img(concat_img)):
    large = nilearn.image.largest_connected_component_img(cur_img)
    print(large)


<class 'nibabel.nifti1.Nifti1Image'>
data shape (61, 73, 61)
affine:
[[  -3.   -0.   -0.   90.]
 [  -0.    3.   -0. -126.]
 [   0.    0.    3.  -72.]
 [   0.    0.    0.    1.]]
metadata:
<class 'nibabel.nifti1.Nifti1Header'> object, endian='<'
sizeof_hdr      : 348
data_type       : b''
db_name         : b''
extents         : 0
session_error   : 0
regular         : b'r'
dim_info        : 48
dim             : [ 3 61 73 61  1  1  1  1]
intent_p1       : 0.0
intent_p2       : 0.0
intent_p3       : 0.0
intent_code     : none
datatype        : float32
bitpix          : 32
slice_start     : 0
pixdim          : [-1.  3.  3.  3.  1.  1.  1.  1.]
vox_offset      : 0.0
scl_slope       : nan
scl_inter       : nan
slice_end       : 60
slice_code      : unknown
xyzt_units      : 10
cal_max         : 1.0
cal_min         : 0.0
slice_duration  : 0.0
toffset         : 0.0
glmax           : 0
glmin           : 0
descrip         : b''
aux_file        : b''
qform_code      : scanner
sform_code      : sc

KeyboardInterrupt: 

In [90]:
concat_img.shape

(61, 73, 61, 176)

In [101]:
qq= nilearn.image.clean_img(concat_img, t_r=2.0)

In [102]:
qq.shape

(61, 73, 61, 176)

In [23]:
labels = []  # 1 if ADHD, 0 if control
img_file = []
#mask_files = []
img_label = []
list_concats = []

for func_file, confound_file, phenotypic in zip(
        adhd_data.func, adhd_data.confounds, adhd_data.phenotypic):
    
    #mask_files.append(mask)
    
    concat_img = nilearn.image.concat_imgs(func_file)
    
    list_concats.append(concat_img)
    img_file.append(func_file)
    labels.append(phenotypic['adhd'])
    img_label.append((func_file, phenotypic['adhd'] ))

In [32]:
list_concats[22].shape

(61, 73, 61, 236)

In [34]:
x = nib.load(func_file)

In [35]:
x.get_fdata().shape

(61, 73, 61, 176)

In [24]:
concat_img.shape

(61, 73, 61, 176)

In [5]:
def resize_image(img):
        FMRI_data = np.asanyarray(nib.load(img).dataobj)
        fmri_t = np.transpose(FMRI_data,axes=[3, 0, 1, 2])
        time_length= 52
        new_depth = 31
        new_width = 42
        new_height = 31
        old_depth = fmri_t.shape[1]
        old_width = fmri_t.shape[2]
        old_height = fmri_t.shape[3]
        ft = time_length/fmri_t.shape[0]
        fx = new_depth/old_depth
        fy = new_width/old_width
        fz = new_height/old_height
        fmri_resize = ndimage.zoom(fmri_t, (ft,fx,fy,fz) ,order=1)
        return fmri_resize[:,:,:,:,None]

In [6]:
b = resize_image('/home/jupyter/nilearn_data/adhd/data/0027037/0027037_rest_tshift_RPI_voreg_mni.nii.gz')

In [7]:
b.shape

(52, 31, 42, 31, 1)

In [14]:
np.asanyarray(b, dtype="int32").shape

(52, 31, 42, 31, 1)

In [11]:
import math
from numbers import Number
from nilearn import plotting

def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))


def _float_feature(value):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))


def serialize_example(img: np.ndarray, label: Number) -> tf.train.Example:
    # convert to numpy arraw
    #nii0 = np.asanyarray(nib.load(img).dataobj)
    
    nii0 = resize_image(img)
    nii1 = np.asanyarray(nii0, dtype="int32")
    #nii = nii0[time_index, ..., 0]
    # rescale to 0-1
    # if you want to use tf.image.per_image_standardization(), this would be the place to do so
    # instead of the rescaling
    #nii = (nii0 - nii0.min())/(nii0.max() - nii0.min()).astype(np.float32) 
    nii = tf.image.per_image_standardization(nii1)
    
    #if nii.shape[-1] < 261:
        # Pad the time dimension with zeros
    #    N = 261 - nii.shape[-1]
    #    pad_width = ((0, N), (0, 0), (0, 0), (0, 0))
    #    nii = np.pad(nii, pad_width, mode='constant', constant_values=(0))
    
    
    feature = {
        'label': _float_feature(label),
        'image_raw': _bytes_feature(tf.io.serialize_tensor(nii).numpy())
    }

    return tf.train.Example(features=tf.train.Features(feature=feature))


def write_records(niis, labels, n_per_record: int, outfile: str) -> None:
    """
    store list of niftis (and associated label) into tfrecords for use as dataset
    """
    n_niis = len(niis)
    n_records = math.ceil(len(niis) / n_per_record)

    for i, shard in enumerate(range(0, n_niis, n_per_record)):
        print(f"writing record {i} of {n_records-1}")
        with tf.io.TFRecordWriter(
                f"{outfile}_{i:0>3}-of-{n_records-1:0>3}.tfrecords", 
            options= tf.io.TFRecordOptions(compression_type="GZIP")
        ) as writer:
            for nii, label in zip(niis[shard:shard+n_per_record], labels[shard:shard+n_per_record]):
                example = serialize_example(img=nii, label=label)
                writer.write(example.SerializeToString())



def parse_1_example(example) -> tf.Tensor:
    X = tf.io.parse_tensor(example['image_raw'], out_type=tf.float32)
    return tf.expand_dims(X, -1), example['label'] 


def decode_example(record_bytes)-> dict:
    example = tf.io.parse_example(
        record_bytes,     
        features = {
          "label": tf.io.FixedLenFeature([], dtype=tf.float32),
          'image_raw': tf.io.FixedLenFeature([], dtype=tf.string)
          }
    )
    return example

def get_batched_dataset(files, batch_size: int = 32, shuffle_size: int=1024) -> tf.data.Dataset:
    dataset = (
        tf.data.Dataset.list_files(files) # note shuffling is on by default
        .flat_map(lambda x: tf.data.TFRecordDataset(x, compression_type="GZIP", num_parallel_reads=8))
        .map(decode_example, num_parallel_calls=tf.data.AUTOTUNE)
        .map(parse_1_example, num_parallel_calls=tf.data.AUTOTUNE)
        .cache()  # remove if all examples don't fit in memory (note interaction with shuffling of files, above)
        .shuffle(shuffle_size)
        .batch(batch_size, num_parallel_calls=tf.data.AUTOTUNE)
        .prefetch(tf.data.AUTOTUNE)
    )
    return dataset



In [12]:
# store example set of image with labels 0-9
# (e.g., put nifti of label MNI152_T1_1mm_brain.nii.gz in the working directory)
mni_nii = ['/home/jupyter/nilearn_data/adhd/data/0010042/0010042_rest_tshift_RPI_voreg_mni.nii.gz']*10

# store examples in each tfrecord. number of examples per record is configurable.
# aim for as many examples as produces files of size > 100M 
write_records(mni_nii, [x for x in range(10)], 10, "tmp")

# read the records back. this will be the list of files generated by write_records()
# a full dataset will have a list with many records
list_of_records=['tmp_000-of-000.tfrecords']
ds = get_batched_dataset(list_of_records, batch_size=2, shuffle_size=10)

# ds can now be passed to model.fit
# but first!!!!! 
# the serialization is a lot, so it is a good idea to verify that the images
# look okay when loaded
(Xs, Ys) = next(ds.as_numpy_iterator())

# (batch_size, )
# order will depend on shuffle (turn off all shuffling to verify order)
Ys.shape

# (batch_size, x_dim, y_dim, z_dim, 1)
Xs.shape

# convert first element in batch to in-memory nibabel.nifti format for display
nii = nib.Nifti1Image(np.squeeze(Xs[0,]), affine=np.eye(4)*2)

# this should look like a typical brain
plotting.plot_anat(nii)

writing record 0 of 0


2023-10-01 09:53:27.786244: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-01 09:53:27.789841: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-01 09:53:27.792416: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

DimensionError: Input data has incompatible dimensionality: Expected dimension is 3D and you provided a 4D image. See https://nilearn.github.io/stable/manipulating_images/input_output.html.

In [ ]:
# store example set of image with labels 0-9
# (e.g., put nifti of label MNI152_T1_1mm_brain.nii.gz in the working directory)
mni_nii = [item[0] for item in img_label]

labels = [item[1] for item in img_label]


# store examples in each tfrecord. number of examples per record is configurable.
# aim for as many examples as produces files of size > 100M 
write_records(mni_nii, labels, 40, "tmp")

# read the records back. this will be the list of files generated by write_records()
# a full dataset will have a list with many records
list_of_records=['tmp_000-of-000.tfrecords']
ds = get_batched_dataset(list_of_records, batch_size=40, shuffle_size=10)


In [ ]:
(Xs, Ys) = next(ds.as_numpy_iterator())

In [ ]:
# (batch_size, )
# order will depend on shuffle (turn off all shuffling to verify order)
Ys.shape

In [ ]:
Ys

In [ ]:

# (batch_size, x_dim, y_dim, z_dim, 1)
Xs.shape

In [ ]:
# Assuming you have already loaded your data into (Xs, Ys)
total_samples = Xs.shape[0]  # or Ys.shape[0], assuming they have the same number of samples

# Define the proportion you want to allocate for training and testing
train_ratio = 0.7  # For an 70-30 split

train_size = int(total_samples * train_ratio)
test_size = total_samples - train_size

# Split the data into training and testing sets
X_train, X_test = Xs[:train_size], Xs[train_size:]
Y_train, Y_test = Ys[:train_size], Ys[train_size:]


In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
# Assuming you have a dataset `ds` and you want to split it into train and test sets

# Calculate the total size of your dataset
total_samples = sum(1 for _ in ds)

# Define the percentage split for training and testing
train_percentage = 0.7  # Adjust this as needed

# Calculate the sizes for train and test datasets
train_size = int(total_samples * train_percentage)
test_size = total_samples - train_size

# Shuffle the dataset
ds = ds.shuffle(buffer_size=total_samples, reshuffle_each_iteration=False)

# Split the dataset into train and test
train_dataset = ds.take(train_size)
test_dataset = ds.skip(train_size)

# Now you can use `train_dataset` and `test_dataset` for training and testing your model


### Loop

In [ ]:
from nilearn.connectome import ConnectivityMeasure

In [ ]:
labels=[]  # 1 if ADHD, 0 if control
resized_img=[]
correlations = []
regions_extracted_img = []

# Initializing ConnectivityMeasure object with kind='correlation'
connectome_measure = ConnectivityMeasure(kind="correlation")

for func_file, confound_file, phenotypic in zip(
        adhd_data.func, adhd_data.confounds, adhd_data.phenotypic):
    
    preprocess = resize_image(func_file)
    timeseries_each_subject = extractor.transform(func_file, confounds=confound_file)
    regions_extracted = extractor.regions_img_
    
    correlation = connectome_measure.fit_transform([timeseries_each_subject])
    
    regions_extracted_img.append(regions_extracted)
    resized_img.append(preprocess)
    correlations.append(correlation)
    labels.append(phenotypic['adhd'])

In [ ]:
print('N control:' ,labels.count(0))
print('N adhd:' ,labels.count(1))

In [ ]:
regions_extracted_img[20].shape

In [ ]:
np.array(nib.load(adhd_data.func[0]).get_fdata()).shape

In [ ]:
# Extracted regions are stored in regions_img_
#regions_extracted_img = extractor.regions_img_
# Each region index is stored in index_
#regions_index = extractor.index_
# Total number of regions extracted
n_regions_extracted = regions_extracted_img[0].shape[-1]

In [ ]:
n_regions_extracted

### mask keeping 3d ?

In [ ]:
resized_img = resize_image(adhd_data['func'][0])
fmri_img= adhd_data['func'][0]

In [ ]:
img_example = nilearn.image.new_img_like(fmri_img , resized_img , affine=img.affine , copy_header=True) 

In [ ]:
func_file

In [ ]:
from nilearn.image import resample_to_img as res
from nilearn.masking import compute_epi_mask
from nilearn.image import iter_img, math_img, concat_imgs, crop_img

In [ ]:
fmri = nib.load(func_file)
mask = nilearn.masking.compute_epi_mask(fmri)
filename = func_file[:-17] + "masked.nii"
masked_list = []
    
for img in iter_img(fmri):
    masked = math_img('img*mask', img = img, mask=mask)
    masked_list.append(masked)
        
final = concat_imgs(masked_list)
crop = crop_img(final)
print(crop.shape)
crop_resampled = res(crop, img_example, interpolation = 'nearest')
print(crop_resampled.shape)  
crop_resampled.to_filename(filename)

In [ ]:
from nilearn.image import load_img

In [ ]:
for func_file, confound_file, phenotypic in zip(
        adhd_data.func, adhd_data.confounds, adhd_data.phenotypic):
    
    fmri = nib.load(func_file)
    mask = nilearn.masking.compute_epi_mask(fmri)
    filename = func_file[:-17] + "masked.nii"
    masked_list = []
    
    for img in iter_img(fmri):
        masked = math_img('img*mask', img = img, mask=mask)
        masked_list.append(masked)
        
    final = concat_imgs(masked_list)
    crop = crop_img(final)
    print(crop.shape)
    crop_resampled = res(crop, img_example, interpolation = 'nearest')
    print(crop_resampled.shape)  
    crop_resampled.to_filename(filename)

### transform the data array back to image

In [ ]:
fmri_img= adhd_data['func'][0]

In [ ]:
new = nilearn.image.new_img_like(img , resized_img[0] , affine=img.affine , copy_header=True) 

In [ ]:
header= new.header

In [ ]:
header.get_zooms()

In [ ]:
np.array(new.get_fdata()).shape

In [ ]:
np.array(resized_img).shape

### train test data

In [ ]:
def get_train_test(X, y, i, verbose=False):
    '''
    Split data into train and test, reshape data, and obtain spatial dimensions.
    
    '''
    
    # Split data into train and test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=i)


    # Reshapes data
    t_shape = np.array(X).shape[1]

    X_train = np.reshape(X_train, (len(X_train), t_shape, 31, 42, 31, 1))
    X_test = np.reshape(X_test, (len(X_test), t_shape, 31, 42, 31, 1))

    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')


    if verbose:
        print(X_train.shape[0], 'train samples')
        print(X_test.shape[0], 'test samples')

    # Converts class vectors to binary class matrices
    y_train = utils.to_categorical(y_train, 2)
    y_test = utils.to_categorical(y_test, 2)

    return X_train, X_test, y_train, y_test, t_shape

In [ ]:
X_train, X_test, y_train, y_test, t_shape = get_train_test(resized_img, labels, i=8, verbose=True)

In [ ]:
X_train.shape

In [ ]:
np.array(resized_img).shape

In [ ]:
y_train.shape

### Model 1

In [ ]:
# Check the shape of data batches from the training dataset
for batch in train_dataset:
    Xs, Ys = batch
    print(f"Xs shape: {Xs.shape}, Ys shape: {Ys.shape}")

# Check the shape of data batches from the test dataset
for batch in test_dataset:
    Xs, Ys = batch
    print(f"Xs shape: {Xs.shape}, Ys shape: {Ys.shape}")


In [ ]:
def model_design(input_shape):
    # Input layer
    inputs = Input(input_shape)
    
    # Convolutional Layer 1
    x = TimeDistributed(Conv3D(filters=32, kernel_size=(7, 7, 7), strides=(1, 1, 1), activation="relu", padding='same'), name="Conv_Layer_1")(inputs)
    x = TimeDistributed(MaxPool3D(pool_size=(2, 2, 2), strides=(2, 2, 2), name="Max_pool_1"))(x)
    x = BatchNormalization()(x)

    # Convolutional Layer 2
    x = TimeDistributed(Conv3D(filters=64, kernel_size=(5, 5, 5), strides=(1, 1, 1), activation="relu", padding='same'), name="Conv_Layer_2")(x)
    x = TimeDistributed(MaxPool3D(pool_size=(2, 2, 2), strides=(2, 2, 2), name="Max_pool_2"))(x)
    x = BatchNormalization()(x)

    # Convolutional Layer 3
    x = TimeDistributed(Conv3D(filters=128, kernel_size=(3, 3, 3), strides=(1, 1, 1), activation="relu", padding='same'), name="Conv_Layer_3")(x)
    x = TimeDistributed(MaxPool3D(pool_size=(2, 2, 2), strides=(2, 2, 2), name="Max_pool_3"))(x)
    x = BatchNormalization()(x)

    # Convolutional Layer 4
    x = TimeDistributed(Conv3D(filters=254, kernel_size=(3, 3, 3), strides=(1, 1, 1), activation="relu", padding='same'), name="Conv_Layer_4")(x)
    x = TimeDistributed(MaxPool3D(pool_size=(2, 2, 2), strides=(2, 2, 2), name="Max_pool_4"))(x)
    x = BatchNormalization()(x)

    # Flatten Layer
    x = TimeDistributed(Flatten(), name="Flatten_Layer2")(x)

    # Global Average Pooling Layer
    x = GlobalAveragePooling1D()(x)

    # Fully connected layers for classification
    outputs = Dense(units=1, activation="relu")(x)

    model = Model(inputs, outputs, name="3dcnn")
    return model

In [ ]:

# Define your loss and optimizer
loss_fn = tf.keras.losses.BinaryCrossentropy()
optimizer = tf.keras.optimizers.Adam()

# Custom training loop
@tf.function
def train_step(inputs, labels):
    with tf.GradientTape() as tape:
        predictions = model(inputs, training=True)
        loss = loss_fn(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss

# Lists to store training and validation loss
train_loss_history = []
validation_loss_history = []

# Training loop
epochs = 50
for epoch in range(epochs):
    for inputs, labels in train_dataset:
        inputs = tf.expand_dims(inputs, axis=0)  # Add an extra batch dimension
        loss = train_step(inputs, labels)
        train_loss_history.append(loss)
    print(f"Epoch {epoch + 1}, Loss: {loss}")

    # Validation (similar loop for validation)
    validation_losses = []
    for inputs, labels in test_dataset:
        inputs = tf.expand_dims(inputs, axis=0)
        predictions = model(inputs, training=False)
        validation_loss = loss_fn(labels, predictions)
        validation_losses.append(validation_loss)
    validation_loss_history.append(tf.reduce_mean(validation_losses))

    print(f"Epoch {epoch + 1}, Loss: {loss}, Validation Loss: {tf.reduce_mean(validation_losses)}")


# Plot the training and validation loss
plt.figure(figsize=(10, 6))
plt.plot(range(1, epochs + 1), train_loss_history, label='Training Loss', marker='o')
plt.plot(range(1, epochs + 1), validation_loss_history, label='Validation Loss', marker='o')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid()
plt.show()


In [ ]:
def model_design(input_shape):
    # Input layer
    inputs = Input(input_shape)
    
    # Convolutional Layer 1
    x = Conv3D(filters=32, kernel_size=(7, 7, 7), strides=(1, 1, 1), activation="relu", padding='same', name="Conv_Layer_1")(inputs)
    x = TimeDistributed(MaxPool3D(pool_size=(2, 2, 2), strides=(2, 2, 2), name="Max_pool_1"))(x)
    x = BatchNormalization()(x)

    # Convolutional Layer 2
    x = Conv3D(filters=64, kernel_size=(5, 5, 5), strides=(1, 1, 1), activation="relu", padding='same', name="Conv_Layer_2")(x)
    x = TimeDistributed(MaxPool3D(pool_size=(2, 2, 2), strides=(2, 2, 2), name="Max_pool_2"))(x)
    x = BatchNormalization()(x)

    # Convolutional Layer 3
    x = Conv3D(filters=128, kernel_size=(3, 3, 3), strides=(1, 1, 1), activation="relu", padding='same', name="Conv_Layer_3")(x)
    x = TimeDistributed(MaxPool3D(pool_size=(2, 2, 2), strides=(2, 2, 2), name="Max_pool_3"))(x)
    x = BatchNormalization()(x)

    # Convolutional Layer 4
    x = Conv3D(filters=254, kernel_size=(3, 3, 3), strides=(1, 1, 1), activation="relu", padding='same', name="Conv_Layer_4")(x)
    x = TimeDistributed(MaxPool3D(pool_size=(2, 2, 2), strides=(2, 2, 2), name="Max_pool_4"))(x)
    x = BatchNormalization()(x)

    # Flatten Layer
    x = TimeDistributed(Flatten(), name="Flatten_Layer2")(x)

    # Global Average Pooling Layer
    x = GlobalAveragePooling1D()(x)

    # Fully connected layers for classification
    outputs = Dense(units=1, activation="relu")(x)

    model = Model(inputs, outputs, name="3dcnn")
    return model

input_shape = (52, 31, 42, 31, 1)
model = model_design(input_shape)

In [ ]:
print(model.summary())

In [ ]:
input_shape = (40, 52, 31, 42, 31, 1)
model = model_design(input_shape)
print(model.summary())

In [ ]:
plot_model(model, show_shapes=True, show_layer_names=True)

In [ ]:
from keras import backend as K

# Clear the Keras session
K.clear_session()

In [ ]:
# Compile model.
initial_learning_rate = 0.0001
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=100000, decay_rate=0.96, staircase=True
)

model.compile(
    loss="binary_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=lr_schedule),
    metrics=["acc"],
)

In [ ]:
# Train the model, doing validation at the end of each epoch
epochs = 100
history = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=epochs,
    verbose=2
)

In [ ]:
evaluation = model.evaluate(X_test, y_test, batch_size=8)
print('Loss in Test set:      %.02f' % (evaluation[0]))
print('Accuracy in Test set:  %.02f' % (evaluation[1] * 100))

### Seeing results

In [ ]:

# Extract training and validation loss and accuracy
training_loss = history.history['loss']
validation_loss = history.history['val_loss']
training_accuracy = history.history['acc']
validation_accuracy = history.history['val_acc']

# Create a range of epochs
epochs = range(1, len(training_loss) + 1)

# Plot training and validation loss
plt.figure(figsize=(6, 2))
plt.subplot(1, 2, 1)
plt.plot(epochs, training_loss, 'r', label='Training Loss')
plt.plot(epochs, validation_loss, 'b', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Plot training and validation accuracy
plt.subplot(1, 2, 2)
plt.plot(epochs, training_accuracy, 'r', label='Training Accuracy')
plt.plot(epochs, validation_accuracy, 'b', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
from sklearn.metrics import accuracy_score
# Initialize lists to store training and validation scores
train_scores, valid_scores = [], []
train_sizes = [0.1, 0.3, 0.5, 0.7, 0.9]  # Adjust these values as needed

for train_size in train_sizes:
    # Determine the number of samples to use for training
    num_samples = int(len(X_train) * train_size)
    
    # Train the model on the subset of training data
    #model.fit(X_train[:num_samples], y_train[:num_samples], epochs=30, batch_size=4, verbose=0)
    
    # Make predictions on the validation data
    y_pred = model.predict(X_test, batch_size=4)
    
    # Convert predicted probabilities to binary labels
    y_pred_labels = (y_pred > 0.5).astype(int)
    
    # Calculate accuracy
    train_score = accuracy_score(y_train[:num_samples], (model.predict(X_train[:num_samples], batch_size=4) > 0.5).astype(int))
    valid_score = accuracy_score(y_test, y_pred_labels)
    
    train_scores.append(train_score)
    valid_scores.append(valid_score)

# Now, 'train_sizes', 'train_scores', and 'valid_scores' contain the data for creating the learning curve


In [ ]:

# Plot the learning curve
plt.figure(figsize=(4, 2))
plt.plot(train_sizes, train_scores, marker='o', label='Training Score', color='b')
plt.plot(train_sizes, valid_scores, marker='o', label='Validation Score', color='g')

# Add labels and a legend
plt.title('Learning Curve')
plt.xlabel('Training Examples')
plt.ylabel('Accuracy')
plt.legend(loc='best')

# Add grid lines
plt.grid(True)

# Show the plot
plt.show()


In [ ]:

# Initialize lists to store confusion matrices
confusion_matrices = []

# Compute confusion matrix for each label
for label in range(2):
    cm = confusion_matrix(y_true[:, label], y_pred_labels[:, label])
    confusion_matrices.append(cm)


In [ ]:
# Create subplots for each label's confusion matrix
fig, axes = plt.subplots(1, 2, figsize=(12, 5))  # Adjust the number of subplots based on the number of labels

for label, ax, cm in zip(range(2), axes, confusion_matrices):
    im = ax.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    ax.figure.colorbar(im, ax=ax)
    
    # Add annotations to the heatmap
    for i in range(2):
        for j in range(2):
            ax.text(j, i, str(cm[i, j]), ha='center', va='center', color='black', fontsize=14)

    # Set axis labels and titles
    classes = ['Negative', 'Positive']  # Update class labels if needed
    ax.set(xticks=np.arange(2), yticks=np.arange(2),
           xticklabels=classes, yticklabels=classes,
           title=f'Confusion Matrix (Label {label})',
           xlabel='Predicted', ylabel='True')

# Display the plot
plt.show()


In [ ]:
from sklearn.metrics import hamming_loss

hamming_loss_value = hamming_loss(y_true, y_pred_labels)
print(f'Hamming Loss: {hamming_loss_value}')


In [ ]:
# Assuming you have a trained Keras model named 'model' and a test dataset 'X_test'
#y_pred = model.predict(X_test, batch_size=4)

# 'y_pred' is an array of predicted probabilities; you can convert it to predicted labels
# Assuming you want to threshold at 0.5 for binary classification
#y_pred_labels = (y_pred > 0.5).astype(int)


### Model 2

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import MaxPooling3D
import tensorflow.keras.layers as layers

In [ ]:
from keras import backend as K

# Clear the Keras session
K.clear_session()

In [ ]:
def simple_3d_cnn(input_shape):
    inputs = Input(input_shape)
    
    inputs = keras.Input((input_shape))

    x = layers.Conv3D(filters=8, kernel_size=(7, 7, 7), strides=(1, 1, 1), activation="relu", padding='valid')(inputs)
    x = layers.TimeDistributed(layers.MaxPool3D(pool_size=(2, 2, 2), strides=(2, 2, 2)))(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv3D(filters=16, kernel_size=(5, 5, 5), strides=(1, 1, 1), activation="relu", padding='valid')(x)
    x = layers.TimeDistributed(layers.MaxPool3D(pool_size=(2, 2, 2), strides=(2, 2, 2)))(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv3D(filters=32, kernel_size=(3, 3, 3), strides=(1, 1, 1), activation="relu", padding='valid')(x)
    x = layers.TimeDistributed(layers.MaxPool3D(pool_size=(2, 2, 2), strides=(2, 2, 2)))(x)
    x = layers.BatchNormalization()(x)

    x = layers.TimeDistributed(layers.GlobalAveragePooling3D())(x)
    x = layers.Dense(units=128, activation="sigmoid")(x)
    x = layers.Flatten()(x) 
    x = layers.Dropout(0.3)(x)

    outputs = layers.Dense(units=1, activation="relu")(x)

    simpler_model = Model(inputs, outputs, name="3dcnn")
    return simpler_model

# Create the simpler model
input_shape = (52, 31,42,31,1)
simpler_model = simple_3d_cnn(input_shape)


In [ ]:
# Compile model.
initial_learning_rate = 0.0001
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=100000, decay_rate=0.96, staircase=True
)

simpler_model.compile(
    loss="binary_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=lr_schedule),
    metrics=["acc"],
)

In [ ]:
# Implement early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Implement learning rate scheduling
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-7)


In [ ]:
# Train the model with early stopping and learning rate scheduling
s_model = simpler_model.fit(X_train, y_train, epochs=50, batch_size=8, validation_data=(X_test, y_test), callbacks=[early_stopping, reduce_lr])


In [ ]:
# Evaluate the model
loss, accuracy = simpler_model.evaluate(X_test, y_test, batch_size=4)
print('Loss in Test set:      %.02f' % (evaluation[0]))
print('Accuracy in Test set:  %.02f' % (evaluation[1] * 100))

In [ ]:

# Extract training and validation loss and accuracy
training_loss = s_model.history['loss']
validation_loss = s_model.history['val_loss']
training_accuracy = s_model.history['acc']
validation_accuracy = s_model.history['val_acc']

# Create a range of epochs
epochs = range(1, len(training_loss) + 1)

# Plot training and validation loss
plt.figure(figsize=(6, 2))
plt.subplot(1, 2, 1)
plt.plot(epochs, training_loss, 'r', label='Training Loss')
plt.plot(epochs, validation_loss, 'b', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Plot training and validation accuracy
plt.subplot(1, 2, 2)
plt.plot(epochs, training_accuracy, 'r', label='Training Accuracy')
plt.plot(epochs, validation_accuracy, 'b', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()
